In [1]:
import glob,os
import numpy as np
import pandas as pd

### Read multiple files

In [2]:
path = "OutputIL"

In [3]:
file = glob.glob(os.path.join(path, "*.bed"))

In [4]:
d = []
for f in file:
    d.append(pd.read_table(f))

### Delete empty dataframe

In [5]:
for i in range(len(d))[::-1]:
    if len(d[i]) == 0:
        del d[i]

In [6]:
Data = pd.concat(d)

### Get iM6A sites

#### For positive strand

In [9]:
Positive = Data[Data["strand"] == "+"]

In [10]:
Positive["Start"] = Positive["Start"] + 1
Positive["End"] = Positive["End"] + 1

#### For negative strand

In [13]:
Negative = Data[Data["strand"] == "-"]

In [15]:
Negative["Start"] = Negative["Start"] - 1
Negative["End"] = Negative["End"] - 1

In [17]:
iM6A = Positive.append(Negative)
iM6A = iM6A.reset_index(drop = True)

In [20]:
iM6A = iM6A[["chrom", "Start", "End", "name", "Probabilty", "strand"]]

In [21]:
iM6A["End"] = iM6A["End"] + 1

In [ ]:
iM6A["Start"] = iM6A["Start"]-2
iM6A["End"] = iM6A["End"]+2

In [ ]:
iM6A.to_csv("Temp/RAC.bed", sep="\t", header=False, index=False)

In [ ]:
!bedtools getfasta -s -bed Temp/RAC.bed -fi /home/morgenlefay/Reference/Fasta/mm10.fa -fo Temp/RAC.txt

In [ ]:
from Bio import SeqIO

In [ ]:
meta=[]
sequence=[]
seq = ("Temp/RAC.txt")
for seq_record in SeqIO.parse(seq, "fasta"):
    meta.append(str(seq_record.id))
    sequence.append(str(seq_record.seq))

In [ ]:
Fasta = pd.DataFrame(data ={'Meta':meta,'Motif':sequence})

In [ ]:
Result = pd.merge(iM6A, Fasta, left_index=True, right_index=True, how='outer')

In [ ]:
Result["Motif"] = Result["Motif"].str.upper()

In [ ]:
Result = Result[(Result["Motif"]=="AAACA") | (Result["Motif"]=="AAACC") | (Result["Motif"]=="AAACG") | (Result["Motif"]=="AAACT") | \
                (Result["Motif"]=="AGACA") | (Result["Motif"]=="AGACC") | (Result["Motif"]=="AGACG") | (Result["Motif"]=="AGACT") | \
                (Result["Motif"]=="CAACA") | (Result["Motif"]=="CAACC") | (Result["Motif"]=="CAACG") | (Result["Motif"]=="CAACT") | \
                (Result["Motif"]=="CGACA") | (Result["Motif"]=="CGACC") | (Result["Motif"]=="CGACG") | (Result["Motif"]=="CGACT") | \
                (Result["Motif"]=="GAACA") | (Result["Motif"]=="GAACC") | (Result["Motif"]=="GAACG") | (Result["Motif"]=="GAACT") | \
                (Result["Motif"]=="GGACA") | (Result["Motif"]=="GGACC") | (Result["Motif"]=="GGACG") | (Result["Motif"]=="GGACT") | \
                (Result["Motif"]=="TAACA") | (Result["Motif"]=="TAACC") | (Result["Motif"]=="TAACG") | (Result["Motif"]=="TAACT") | \
                (Result["Motif"]=="TGACA") | (Result["Motif"]=="TGACC") | (Result["Motif"]=="TGACG") | (Result["Motif"]=="TGACT")]

In [ ]:
Result = Result.reset_index(drop = True)

In [ ]:
Result.drop(["Meta", "Motif"], axis=1, inplace=True)

In [ ]:
iM6A = Result
iM6A["Start"] = iM6A["Start"]+2
iM6A["End"] = iM6A["End"]-2

In [ ]:
iM6A

In [ ]:
iM6A = iM6A.sort_values(by=["chrom","Start"], axis=0, ascending=True)
iM6A = iM6A.reset_index(drop = True)

In [ ]:
iM6A.to_csv("iM6A_mouse_RAC_IL.csv",index=0)